In [7]:
# Regular data science libraries
import numpy as np
import pandas as pd
import json
import os
import pickle
import random


# DNN libraries
import torch
import torch.nn as nn
import torch.nn.functional as F


# Import utility functions from other files
from utils import generate_model, dp, generate_offline_data
from env import Agent, Environment


# Plotting libraries
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import scienceplots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.style.use('ieee')


# Clustering libraries
from sklearn.decomposition import PCA
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import SIMPLE_SAMPLES
from scipy.stats import wasserstein_distance

# Set the seeds
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)


In [10]:
import gym
import ale_py
import d4rl_atari

env = gym.make('seaquest-mixed-v0') # -v{0, 1, 2, 3, 4} for datasets with the other random seeds
# env = gym.make('ALE/Breakout-v5')

# interaction with its environment through dopamine-style Atari wrapper
observation = env.reset() # observation.shape == (84, 84)
observation, reward, terminal, info = env.step(env.action_space.sample())

# dataset will be automatically downloaded into ~/.d4rl/datasets/[GAME]/[INDEX]/[EPOCH]
dataset = env.get_dataset()


ModuleNotFoundError: No module named 'd4rl_atari'

In [4]:
# print(dataset['observations'])
print(dataset['observations'].shape) # observation data in (1000000, 1, 84, 84)
print(type(dataset['observations']))

(1000000, 1, 84, 84)
<class 'numpy.ndarray'>


In [5]:
print(np.unique(dataset['actions'])) # action data in (1000000,)
print(dataset['actions'].shape) # observation data in (1000000, )
print(type(dataset['actions']))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
(1000000,)
<class 'numpy.ndarray'>


In [6]:
print(np.unique(dataset['rewards'])) # reward data in (1000000,)
print(dataset['rewards'].shape) # observation data in (1000000, )
print(type(dataset['rewards']))

[0. 1.]
(1000000,)
<class 'numpy.ndarray'>


In [7]:
print(np.unique(dataset['terminals'])) # terminal flags in (1000000,)
print(dataset['terminals'].shape) # observation data in (1000000, )
print(type(dataset['terminals']))

[0 1]
(1000000,)
<class 'numpy.ndarray'>


In [2]:
from decision_transformer_atari import GPTConfig, GPT

In [14]:
vocab_size = 4
block_size = 90
model_type = "reward_conditioned"
timesteps = 2654

mconf = GPTConfig(
    vocab_size,
    block_size,
    n_layer=6,
    n_head=8,
    n_embd=128,
    model_type=model_type,
    max_timestep=timesteps,
)
model = GPT(mconf)

checkpoint_path = "checkpoints/Breakout_123.pth"  # or Pong, Qbert, Seaquestcheckpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint)


EOFError: Ran out of input

In [ ]:
# checkpoints? 
# env?
# where are parameter weights? 

In [15]:
model

GPT(
  (tok_emb): Embedding(4, 128)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU()
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2